# Airbnb Data Preprocessing
This notebook is used to merge the airbnb raw files in AirDNA.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
from scipy.stats import mode
np.random.seed(123)
import os
ROOT = os.getcwd()  # Get the current working directory
DATA = os.path.join(ROOT, "AirDNA")
TABLES = os.path.join(ROOT, "Tables")
print(ROOT) 

/Users/zhangshuyi/Desktop/Airbnb_Analysis


In [2]:
# Axuiliary functions
def replace_space_parentheses(df):
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.replace("(", "")
    df.columns = df.columns.str.replace(")", "")
    return df

In [3]:
# Read data
airdna1_df, airdna2_df = pd.read_csv(os.path.join(DATA, "AirDNA1.csv")), pd.read_csv(os.path.join(DATA, "AirDNA2.csv"))

dtypes = {
    "Property_ID": "int32",
    "Airbnb_Property_ID": "int32",
    "ADR_USD": "float32",
    "Price_USD": "float32",
    "Bathrooms": "float32",
    "Bedrooms": "float32",
    "Max_Guests": "int16",
    "Minimum_Stay": "int16",
    "Number_of_Photos": "int16",
    "Number_of_Reviews": "int32",
    "Occupancy_Rate": "float32",
    "Overall_Rating": "float32",
    "Revenue_USD": "float32",
    "Revenue_Potential_USD": "float32",
    "Security_Deposit_USD": "float32",
    "Zipcode": "object",  # Keep categorical columns as strings
}

#chunksize = 100000
#chunk_list = []
#for chunk in pd.read_csv(os.path.join(DATA, "AirDNA3.csv"), dtype=dtypes, parse_dates=["Date"], chunksize=chunksize):
#    chunk_list.append(chunk)
#airdna3_df = pd.concat(chunk_list, ignore_index=True)
#del chunk_list
# airdna3_df = dd.read_csv(os.path.join(DATA, "AirDNA3.csv"), dtype=dtypes, parse_dates=["Date"], assume_missing=True, 
#                          low_memory=False)
airdna3_df = pd.read_csv(os.path.join(DATA, "AirDNA3.csv"), usecols=['Property ID', 'Date', 'Status', 'Price (USD)',
    'Airbnb Property ID'])
print(airdna3_df.columns)
# airdna_clearned_df = pd.read_csv(os.path.join(DATA, "Cleaned_AirDNA.csv"))
# sort the data by Date and Propety ID
airdna1_df = replace_space_parentheses(airdna1_df)
airdna2_df = replace_space_parentheses(airdna2_df)
airdna3_df = replace_space_parentheses(airdna3_df)


# Deal with date
airdna2_df.rename(columns={"Reporting_Month":"Date"}, inplace=True)
airdna2_df["Date"] = pd.to_datetime(airdna2_df["Date"])
# airdna2_df = airdna2_df.sort_values(by=["Property_ID","Date"])

# airdna3_df = airdna3_df.sort_values(by=["Property_ID","Date"])


/var/folders/qf/_z697sgn3jx3215w7ykp1p1r0000gn/T/ipykernel_2648/4089915124.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  airdna1_df, airdna2_df = pd.read_csv(os.path.join(DATA, "AirDNA1.csv")), pd.read_csv(os.path.join(DATA, "AirDNA2.csv"))


Index(['Property ID', 'Date', 'Status', 'Price (USD)', 'Airbnb Property ID'], dtype='object')


In [4]:
print(airdna1_df.columns)
# Are the Property_IDs unique? How many of them are replicated in the data?
print(airdna1_df["Property_ID"].nunique(), airdna1_df["Property_ID"].count())
print(airdna1_df["Airbnb_Property_ID"].nunique(), airdna1_df["Airbnb_Property_ID"].count())
print(airdna1_df.describe())
airdna1_df.head()

Index(['Property_ID', 'Airbnb_Property_ID', 'Airbnb_HOST_ID',
       'VRBO_Property_ID', 'VRBO_Host_ID', 'Listing_Title', 'Property_Type',
       'Real_Estate_Property_Type', 'Listing_Type', 'Property_Manager',
       'Host_Type', 'Country', 'State', 'City', 'Zipcode', 'Neighborhood',
       'Metropolitan_Statistical_Area', 'AirDNA_Market', 'AirDNA_Submarket',
       'Latitude', 'Longitude', 'Exact_Location', 'Location_Type',
       'Price_Tier', 'Bedrooms', 'Bathrooms', 'Max_Guests',
       'Cancellation_Policy', 'Minimum_Stay', 'Created_Date',
       'Last_Scraped_Date', 'Last_Calendar_Update', 'Response_Rate',
       'Response_Time', 'Currency_USD', 'ADR_USD', 'Revenue_LTM_USD',
       'Revenue_Potential_LTM_USD', 'Occupancy_Rate_LTM',
       'Number_of_Bookings_LTM', 'Number_of_Reviews', 'Security_Deposit_USD',
       'Cleaning_Fee_USD', 'Extra_People_Fee', 'Published_Nightly_Rate_USD',
       'Published_Monthly_Rate_USD', 'Published_Weekly_Rate_USD',
       'Count_Reservation_Days

,Property_ID,Airbnb_Property_ID,Airbnb_HOST_ID,VRBO_Property_ID,VRBO_Host_ID,Listing_Title,Property_Type,Real_Estate_Property_Type,Listing_Type,Property_Manager,...,Has_Parking,Amenities,Currency_Native,ADR_Native,Revenue_LTM_Native,Revenue_Potential_LTM_Native,Cleaning_Fee_Native,Published_Nightly_Rate_Native,Published_Monthly_Rate_Native,Published_Weekly_Rate_Native
0,abnb_45439051,4.543905e+07,367546574.0,NaN,NaN,Chain O''Lakes Lakeside Getaway,House,House/villa,Entire home/apt,NaN,...,True,"[""hot_water"",""essentials"",""bed_linens"",""dryer""...",USD,NaN,NaN,NaN,150.0,NaN,NaN,NaN
1,abnb_48569951,4.856995e+07,342643084.0,NaN,NaN,Homey place just for you | 1BR in Evanston,Serviced apartment,Apt/Condo/Loft,Entire home/apt,Preferred Corporate Housing (PCH),...,False,"[""pool"",""wireless_internet"",""kitchen"",""gym"",""l...",USD,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,abnb_8267501,8.267501e+06,1779158.0,NaN,NaN,Private room next to subway stop,Apartment,Apt/Condo/Loft,Private room,NaN,...,False,"[""kitchen"",""internet"",""heating"",""washer"",""drye...",USD,NaN,NaN,NaN,10.0,68.0,1280.0,428.0
3,abnb_13209354,1.320935e+07,71964683.0,NaN,NaN,Modern Studio,Apartment,Apt/Condo/Loft,Entire home/apt,NaN,...,False,"[""kitchen"",""heating"",""ac"",""washer"",""dryer"",""wi...",USD,NaN,NaN,NaN,15.0,95.0,2660.0,665.0
4,abnb_1151067740489329223,1.151068e+18,328191065.0,NaN,NaN,Aurora White House,House,House/villa,Entire home/apt,NaN,...,True,"[""ac"",""dedicated_workspace"",""free_parking"",""in...",USD,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [5]:
print(airdna2_df.columns)
print(airdna2_df.describe())
print(airdna2_df["Property_ID"].nunique(), airdna2_df["Property_ID"].count())
print(airdna2_df["Airbnb_Property_ID"].nunique(), airdna2_df["Airbnb_Property_ID"].count())
airdna2_df.head()

Index(['Property_ID', 'Property_Type', 'Listing_Type', 'Bedrooms', 'Date',
       'Occupancy_Rate', 'Currency', 'Revenue_USD', 'Revenue_Potential_USD',
       'ADR_USD', 'Number_of_Reservations', 'Reservation_Days',
       'Available_Days', 'Blocked_Days', 'Country', 'State', 'City', 'Zipcode',
       'Neighborhood', 'Metropolitan_Statistical_Area', 'Latitude',
       'Longitude', 'Active', 'Scraped_During_Month', 'Airbnb_Property_ID',
       'Airbnb_Host_ID', 'Vrbo_Property_ID', 'Vrbo_Host_ID',
       'Property_Manager', 'Revenue_Native', 'ADR_Native',
       'Revenue_Potential_Native'],
      dtype='object')
           Bedrooms                           Date  Occupancy_Rate  \
count  2.759647e+06                        2763758    2.763758e+06   
mean   1.680840e+00  2020-06-25 06:22:50.969816320    2.425868e-01   
min    0.000000e+00            2014-05-01 00:00:00    0.000000e+00   
25%    1.000000e+00            2018-05-01 00:00:00    0.000000e+00   
50%    1.000000e+00            2

,Property_ID,Property_Type,Listing_Type,Bedrooms,Date,Occupancy_Rate,Currency,Revenue_USD,Revenue_Potential_USD,ADR_USD,...,Active,Scraped_During_Month,Airbnb_Property_ID,Airbnb_Host_ID,Vrbo_Property_ID,Vrbo_Host_ID,Property_Manager,Revenue_Native,ADR_Native,Revenue_Potential_Native
0,abnb_11860623,Bed & Breakfast,Private room,1.0,2016-03-01,0.000000,usd,0.0,0.00,0.00,...,True,True,11860623.0,32907860.0,NaN,NaN,NaN,0.00,0.00,0.00
1,abnb_11860623,Bed & Breakfast,Private room,1.0,2016-04-01,0.222222,usd,276.0,330.84,138.00,...,True,True,11860623.0,32907860.0,NaN,NaN,NaN,275.48,137.74,330.32
2,abnb_11860623,Bed & Breakfast,Private room,1.0,2016-05-01,0.419355,usd,1722.0,1722.00,132.46,...,True,True,11860623.0,32907860.0,NaN,NaN,NaN,1722.72,132.52,1722.72
3,abnb_11860623,Bed & Breakfast,Private room,1.0,2016-06-01,0.633333,usd,2372.0,2372.00,124.84,...,True,True,11860623.0,32907860.0,NaN,NaN,NaN,2374.56,124.98,2374.56
4,abnb_11860623,Bed & Breakfast,Private room,1.0,2016-07-01,0.354839,usd,1438.0,1438.00,130.73,...,True,True,11860623.0,32907860.0,NaN,NaN,NaN,1437.54,130.69,1437.54


In [6]:
# Display the dates covered by the airdna3_df Date column. So does the date covered in airdna2_df
print(airdna3_df["Date"].min(), airdna3_df["Date"].max())
print(airdna2_df["Date"].min(), airdna2_df["Date"].max())
# Also, check the date frequency of the two dataframes
print(airdna3_df["Date"].value_counts())
print(airdna2_df["Date"].value_counts())

2014-10-01 2024-10-31
2014-05-01 00:00:00 2024-10-01 00:00:00
Date
2024-10-31    32921
2024-10-16    32921
2024-10-01    32921
2024-10-02    32921
2024-10-03    32921
              ...  
2014-10-28     8800
2014-10-29     8800
2014-10-02     8800
2014-10-31     8800
2014-10-01     8800
Name: count, Length: 3684, dtype: int64
Date
2024-10-01    32921
2024-09-01    32680
2024-08-01    32345
2024-03-01    31938
2024-07-01    31929
              ...  
2014-09-01     5736
2014-08-01     2362
2014-07-01     1542
2014-06-01     1335
2014-05-01     1152
Name: count, Length: 126, dtype: int64


In [ ]:
'''
airdna3_df["Date"] = pd.to_datetime(airdna3_df["Date"])
# Only keep data from 2020 to 2024 in airdna3_df
airdna3_5years_df = airdna3_df[airdna3_df["Date"].dt.year >= 2020]
#airbnb_5years_df = main_df[main_df['Date'] >= '2023-01-01']
print(airdna3_5years_df.columns)
'''

Index(['Property_ID', 'Date', 'Status', 'Price_USD', 'Airbnb_Property_ID'], dtype='object')


In [ ]:
'''
airdna3_5years_df['Price_USD'] = airdna3_5years_df['Price_USD'].round(4)
print(airdna3_df.head())
print(airdna3_df.describe())
airdna3_df.to_csv(os.path.join(DATA, "AirDNA3_5years.csv"), index=False)
'''

/var/folders/qf/_z697sgn3jx3215w7ykp1p1r0000gn/T/ipykernel_2581/1435685989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airdna3_5years_df['Price_USD'] = airdna3_5years_df['Price_USD'].round(4)


    Property_ID       Date Status  Price_USD  Airbnb_Property_ID
0  abnb_5917075 2014-10-01      A       70.0           5917075.0
1  abnb_5917287 2014-10-01      A      100.0           5917287.0
2  abnb_8192818 2014-10-01      A      160.0           8192818.0
3  abnb_8192987 2014-10-01      A       90.0           8192987.0
4  abnb_6634135 2014-10-01      A       80.0           6634135.0
                                Date     Price_USD  Airbnb_Property_ID
count                       83753663  8.336142e+07        7.370070e+07
mean   2020-07-20 04:25:37.226731776  1.840856e+02        1.148631e+17
min              2014-10-01 00:00:00  0.000000e+00        1.936000e+03
25%              2018-05-22 00:00:00  6.800000e+01        1.328214e+07
50%              2020-08-21 00:00:00  1.100000e+02        2.465854e+07
75%              2022-12-10 00:00:00  1.900000e+02        4.384919e+07
max              2024-10-31 00:00:00  9.999990e+05        1.276452e+18
std                              NaN  1.06

In [ ]:
'''
# Data Merging: Use AirDNA3_5years as the main file and merge the other two files with it. If there are any Property_ID that is in the other two files but not in AirDNA3_5years, we will drop them.
main_5years_df = airdna3_df.sort_values(by=["Property_ID","Date"]).copy()
cols_to_use = airdna1_df.columns.difference(main_5years_df.columns).to_list()
main_5years_df = pd.merge(main_5years_df, airdna1_df[cols_to_use + ['Property_ID','Airbnb_Property_ID']],
                    on=['Property_ID','Airbnb_Property_ID'], how="left")
main_5years_df['Date'] = main_5years_df['Date'].dt.to_timestamp()
#main_df['Date'] = pd.to_datetime(main_df['Date'])

cols_to_use = airdna2_df.columns.difference(main_5years_df.columns).to_list()
main_5years_df = pd.merge(main_5years_df, airdna2_df[cols_to_use + ['Property_ID','Airbnb_Property_ID',"Date"]],
                    on=['Property_ID', 'Airbnb_Property_ID',"Date"], how="left")
print(main_5years_df.shape)
cols_to_keep = ['Property_ID', 'Date', 'Status', 'Price_USD',
        'Airbnb_Property_ID','ADR_USD','Airbnb_Accuracy_Rating',
       'Airbnb_Checkin_Rating', 'Airbnb_Cleanliness_Rating',
       'Airbnb_Communication_Rating', 'Airbnb_HOST_ID',
       'Airbnb_Location_Rating', 'Airbnb_Superhost', 'Airbnb_Value_Rating',
       'Amenities', 'Bathrooms', 'Bedrooms', 'City',
       'Cleaning_Fee_USD', 'Count_Available_Days_LTM', 'Count_Reservation_Days_LTM', 'Country',
       'Created_Date', 'Currency_USD', 'Extra_People_Fee',
       'Has_Air_Con', 'Has_Gym', 'Has_Hot_Tub', 'Has_Kitchen', 'Has_Parking',
       'Has_Pool', 'Host_Type', 'Last_Calendar_Update',
       'Latitude', 'Longitude', 'Listing_Type',
       'Location_Type', 'Max_Guests',
       'Metropolitan_Statistical_Area', 'Minimum_Stay', 'Neighborhood', 'Number_of_Photos', 'Number_of_Reviews',
       'Occupancy_Rate', 'Overall_Rating', 'Pets_Allowed', 'Price_Tier',
       'Published_Monthly_Rate_USD', 'Published_Nightly_Rate_USD', 'Published_Weekly_Rate_USD',
       'Response_Rate', 'Response_Time', 'Revenue_USD', 'Revenue_Potential_USD','Security_Deposit_USD', 'Zipcode']

main_5years_df = main_5years_df[cols_to_keep]
main_5years_df.to_csv(os.path.join(DATA, "Merged_AirDNA_5years.csv"), index=False)
'''

In [ ]:
'''
# Sample 50 random unique Property_IDs from main_df
unique = main_5years_df["Property_ID"].unique()
selected = np.random.choice(unique, 50, replace=False)
sample_df = main_5years_df[main_5years_df["Property_ID"].isin(selected)]
sample_df.to_csv(os.path.join(DATA, "Sample_AirDNA_5years.csv"), index=False)
'''

In [7]:
# Collapse the airbnb3_df data to the monthly level
# First, convert date to month
airdna3_df["Date"] = pd.to_datetime(airdna3_df["Date"])
airdna3_df["Date"] = airdna3_df["Date"].dt.to_period("M")

In [10]:
#def most_frequent(x):
#    return x.mode().iloc[0] if not x.mode().empty else None  # Pandas mode() for categorical data

#airdna3_df = airdna3_df.groupby(["Property_ID", "Airbnb_Property_ID", "Date"]).agg({
#    "Price_USD": "mean",   # Compute mean for price
#    "Status": most_frequent  # Keep most frequent 'Status'
#}).reset_index()

airdna3_df = airdna3_df.groupby(["Property_ID",'Airbnb_Property_ID', "Date"]).agg({"Price_USD":"mean"}).reset_index()
airdna3_df['Price_USD'] = airdna3_df['Price_USD'].round(4)
print(airdna3_df.head())
print(airdna3_df.describe())
airdna3_df.to_csv(os.path.join(DATA, "AirDNA3_monthly.csv"), index=False)

     Property_ID  Airbnb_Property_ID     Date  Price_USD
0  abnb_10000059          10000059.0  2015-12    60.0000
1  abnb_10000059          10000059.0  2016-01    60.0000
2  abnb_10000059          10000059.0  2016-02    60.3448
3  abnb_10000059          10000059.0  2016-03    61.9355
4  abnb_10000059          10000059.0  2016-04    60.0000
       Airbnb_Property_ID     Price_USD
count        2.420456e+06  2.419974e+06
mean         1.146563e+17  1.693423e+02
std          2.875976e+17  4.727815e+02
min          1.936000e+03  0.000000e+00
25%          1.328152e+07  6.492860e+01
50%          2.464350e+07  1.050000e+02
75%          4.383368e+07  1.790000e+02
max          1.276452e+18  1.357186e+05


In [12]:
# read AirDNA3_monthly.csv, save it as airbnb3_df
airdna3_df = pd.read_csv(os.path.join(DATA, "AirDNA3_monthly.csv"))

In [10]:
# mixed_type_cols = []
# for col in airdna3_df.columns:
#     if airdna3_df[col].dtype == "object":
#         mixed_type_cols.append(col)
# safe_cols = [col for col in airdna3_df.columns if col not in mixed_type_cols]

# print(airdna3_df.columns)
# print(airdna3_df[safe_cols].describe().compute())
# print(airdna3_df["Property_ID"].nunique().compute(), airdna3_df["Property_ID"].count().compute())
# print(airdna3_df.head())
# print (airdna3_df.columns)

In [ ]:
# Data Merging: use AirDNA3 as the main file and merge the other two files with it. If there are any Property_ID that is in the other two files but not in AirDNA3, drop them.
main_df = airdna3_df.sort_values(by=["Property_ID","Date"]).copy()
cols_to_use = airdna1_df.columns.difference(main_df.columns).to_list()
main_df = pd.merge(main_df, airdna1_df[cols_to_use + ['Property_ID','Airbnb_Property_ID']],
                    on=['Property_ID','Airbnb_Property_ID'], how="left")
#main_df['Date'] = main_df['Date'].dt.to_timestamp()
main_df['Date'] = pd.to_datetime(main_df['Date'])

cols_to_use = airdna2_df.columns.difference(main_df.columns).to_list()
main_df = pd.merge(main_df, airdna2_df[cols_to_use + ['Property_ID','Airbnb_Property_ID',"Date"]],
                    on=['Property_ID', 'Airbnb_Property_ID',"Date"], how="left")
print(main_df.shape)
cols_to_keep = ['Property_ID', 'Date', 'Price_USD',
        'Airbnb_Property_ID','ADR_USD','Airbnb_Accuracy_Rating',
       'Airbnb_Checkin_Rating', 'Airbnb_Cleanliness_Rating',
       'Airbnb_Communication_Rating', 'Airbnb_HOST_ID',
       'Airbnb_Location_Rating', 'Airbnb_Superhost', 'Airbnb_Value_Rating',
       'Amenities', 'Bathrooms', 'Bedrooms', 'City',
       'Cleaning_Fee_USD', 'Count_Available_Days_LTM', 'Count_Reservation_Days_LTM', 'Country',
       'Created_Date', 'Currency_USD', 'Extra_People_Fee',
       'Has_Air_Con', 'Has_Gym', 'Has_Hot_Tub', 'Has_Kitchen', 'Has_Parking',
       'Has_Pool', 'Host_Type', 'Last_Calendar_Update',
       'Latitude', 'Longitude', 'Listing_Type',
       'Location_Type', 'Max_Guests',
       'Metropolitan_Statistical_Area', 'Minimum_Stay', 'Neighborhood', 'Number_of_Photos', 'Number_of_Reviews',
       'Occupancy_Rate', 'Overall_Rating', 'Pets_Allowed', 'Price_Tier',
       'Published_Monthly_Rate_USD', 'Published_Nightly_Rate_USD', 'Published_Weekly_Rate_USD',
       'Response_Rate', 'Response_Time', 'Revenue_USD', 'Revenue_Potential_USD','Security_Deposit_USD', 'Zipcode']

main_df = main_df[cols_to_keep]
main_df.to_csv(os.path.join(DATA, "Merged_AirDNA.csv"), index=False)

(2420456, 97)


In [16]:
# Sample 50 random unique Property_IDs from main_df
unique = main_df["Property_ID"].unique()
selected = np.random.choice(unique, 50, replace=False)
sample_df = main_df[main_df["Property_ID"].isin(selected)]
sample_df.to_csv(os.path.join(DATA, "Sample_AirDNA.csv"), index=False)